## Import library & facenet-pytorch model

In [ ]:
!git clone https://github.com/timesler/facenet-pytorch.git
!mv ./facenet-pytorch/models .
from models.inception_resnet_v1 import InceptionResnetV1

Cloning into 'facenet-pytorch'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 1181 (delta 16), reused 10 (delta 2), pack-reused 1143
Receiving objects: 100% (1181/1181), 22.61 MiB | 3.77 MiB/s, done.
Resolving deltas: 100% (566/566), done.


In [ ]:
! ls ./drive/My\ Drive/Colab\ Notebooks/E4P2S4_Face_reco/Dataset/

'Donald Trump'	'Emma Watson'	   'Narendra Modi'    'Unknow Face'
'Elon Musk'	'Geoffrey Hinton'  'Priyanka Chopra'  'Will Smith'
'Emma Stone'	'Hrithik Roshan'   'Tom Cruise'


In [ ]:
from torch import nn, optim, as_tensor
import torchvision
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.nn.init import *
from torchvision import transforms, utils, datasets, models
from models.inception_resnet_v1 import InceptionResnetV1
import cv2
from PIL import Image
from pdb import set_trace
import time
import copy
from pathlib import Path
import os
import sys
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage import io, transform
from tqdm import trange, tqdm
import csv
import glob
import dlib
import pandas as pd
import numpy as np
from torchsummary import summary


In [ ]:
num_classes = 11  

## Model

In [ ]:
device = "cuda"
model = InceptionResnetV1(pretrained='vggface2', classify=True, num_classes = num_classes).to(device)
model.logits.out_features = num_classes
# summary(model,(3,224,224))

In [ ]:
total_child = len(list(model.children()))
ct = 0
print("\nFreezed layer except below childs/layers \n")

for child in model.children():
  ct += 1
  if ct < total_child - 5:
    for param in child.parameters():
        param.requires_grad = False
  else:
    print(child)

# https://discuss.pytorch.org/t/how-the-pytorch-freeze-network-in-some-layers-only-the-rest-of-the-training/7088/2 


Freezed layer except below childs/layers 

Block8(
  (branch0): BasicConv2d(
    (conv): Conv2d(1792, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (branch1): Sequential(
    (0): BasicConv2d(
      (conv): Conv2d(1792, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (1): BasicConv2d(
      (conv): Conv2d(192, 192, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1), bias=False)
      (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (2): BasicConv2d(
      (conv): Conv2d(192, 192, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0), bias=False)
      (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
  )
  (conv2d

In [ ]:
for param in model.conv2d_3b.parameters():
  a = 0 #do nothing
print("requires_grad value set to early layer:",param.requires_grad)

requires_grad value set to early layer: False


In [ ]:
# summary(model,(3,224,224))

## Dataset & Transforms

In [ ]:
train_transforms =transforms.Compose([
        transforms.Resize(size=(224, 224),interpolation=2),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])   # Normalized value 
    ])

In [ ]:
!rm -rf .ipynb_checkpoints */.ipynb_checkpoints */*/.ipynb_checkpoints    #delete hidden file before creating dataset

In [ ]:
traindir = './drive/My Drive/Colab Notebooks/E4P2S4_Face_reco/Dataset'

train_dataset = torchvision.datasets.ImageFolder(traindir,transform= train_transforms)

In [ ]:
print("total class", len(train_dataset.classes))
print("class name", train_dataset.classes)

total class 11
class name ['Donald Trump', 'Elon Musk', 'Emma Stone', 'Emma Watson', 'Geoffrey Hinton', 'Hrithik Roshan', 'Narendra Modi', 'Priyanka Chopra', 'Tom Cruise', 'Unknow Face', 'Will Smith']


## Dataloader with sampler

In [ ]:
# Handling imbalance dataset throguh sampler

def make_weights_for_balanced_classes(images, nclasses):                        
    count = [0] * nclasses                                                      
    for item in images:                                                         
        count[item[1]] += 1                                                     
    weight_per_class = [0.] * nclasses                                      
    N = float(sum(count))                                                   
    for i in range(nclasses):               
        weight_per_class[i] = N/float(count[i])                                 
    weight = [0] * len(images)                                              
    for idx, val in enumerate(images):                                          
        weight[idx] = weight_per_class[val[1]]                                  
    return weight  

weights = make_weights_for_balanced_classes(train_dataset.imgs, len(train_dataset.classes))                                                                
weights = torch.DoubleTensor(weights)                                       
sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights))   # provide a weight value for each sample in dataset

#ref: https://discuss.pytorch.org/t/balanced-sampling-between-classes-with-torchvision-dataloader/2703/3

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32  , sampler=sampler)
print("total train dataset:",len(train_dataset))

total train dataset: 1032


## Training

In [ ]:
from my_train import my_train
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9 ) 

train = my_train()

EPOCHS = 21
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer,criterion, epoch)
    if(epoch<3):
        scheduler = StepLR(optimizer, step_size=5, gamma=0.2)
    else:
        scheduler = StepLR(optimizer, step_size=5, gamma=0.5)

  0%|          | 0/33 [00:00<?, ?it/s]

EPOCH: 0


Loss=0.002485201694071293 Batch_id=32 Accuracy=90.21: 100%|██████████| 33/33 [06:11<00:00, 11.25s/it]
  0%|          | 0/33 [00:00<?, ?it/s]

EPOCH: 1


Loss=0.28860339522361755 Batch_id=32 Accuracy=98.84: 100%|██████████| 33/33 [02:05<00:00,  3.79s/it]
  0%|          | 0/33 [00:00<?, ?it/s]

...

Loss=0.0006213889573700726 Batch_id=32 Accuracy=100.00: 100%|██████████| 33/33 [00:19<00:00,  1.69it/s]
  0%|          | 0/33 [00:00<?, ?it/s]

EPOCH: 19


Loss=0.006464610807597637 Batch_id=32 Accuracy=100.00: 100%|██████████| 33/33 [00:18<00:00,  1.83it/s]
  0%|          | 0/33 [00:00<?, ?it/s]

EPOCH: 20


Loss=0.02137087658047676 Batch_id=32 Accuracy=100.00: 100%|██████████| 33/33 [00:17<00:00,  1.86it/s]
